In [52]:
!pip install -U ortools

     |████████████████████████████████| 27.3MB 12.1MB/s eta 0:00:01   |█▍                              | 1.2MB 3.0MB/s eta 0:00:09     |██▏                             | 1.9MB 3.0MB/s eta 0:00:09     |████████████▎                   | 10.5MB 3.0MB/s eta 0:00:06     |█████████████▋                  | 11.6MB 3.0MB/s eta 0:00:06     |██████████████████▉             | 16.1MB 10.6MB/s eta 0:00:02     |██████████████████████▎         | 19.0MB 10.6MB/s eta 0:00:01     |█████████████████████████▎      | 21.6MB 10.6MB/s eta 0:00:01     |████████████████████████████▏   | 24.0MB 12.1MB/s eta 0:00:01     |█████████████████████████████▋  | 25.3MB 12.1MB/s eta 0:00:01     |███████████████████████████████ | 26.5MB 12.1MB/s eta 0:00:01     |███████████████████████████████▊| 27.1MB 12.1MB/s eta 0:00:01
     |████████████████████████████████| 1.2MB 12.9MB/s eta 0:00:01     |███████████████████████▏        | 890kB 12.9MB/s eta 0:00:01


In [5]:
import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp

In [174]:
np.random.seed(42)
samples = 5000

In [7]:
index = ['name', 'cost', 'size', 'handling', 'positioning', 'joining']
names = ['robot' + str(i) for i in range(samples)]
costs = np.random.randint(1000, 10000, size=(samples)).tolist()
handling = np.random.binomial(1, 0.5, samples)
positioning = np.random.binomial(1, 0.5, samples)
joining = np.random.binomial(1, 0.5, samples)
sizes = np.round(np.random.rand((samples)) * 10, 2)

In [35]:
df = pd.DataFrame({'name': names, 'cost': costs, 'size': sizes, 'handling': handling, 'positioning': positioning, 'joining': joining})
df.head(10)

,name,cost,size,handling,positioning,joining
0,robot0,8270,3.49,1,0,0
1,robot1,1860,7.26,0,1,1
2,robot2,6390,8.97,1,0,1
3,robot3,6191,8.87,1,0,0
4,robot4,6734,7.80,0,0,0
5,robot5,7265,6.42,1,0,1
6,robot6,1466,0.84,1,1,0
7,robot7,5426,1.62,1,0,0
8,robot8,6578,8.99,1,1,0
9,robot9,9322,6.06,1,1,1


In [77]:
constraints = {'cost': 20000, 'size': 10.0, 'max_num': 5, 'handling': 8, 'positioning': 5, 'joining': 3}

In [26]:
solver = pywraplp.Solver('simple_lp_programm', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
solver

<ortools.linear_solver.pywraplp.Solver; proxy of <Swig Object of type 'operations_research::MPSolver *' at 0x7f0c659c66f0> >

In [107]:
# create variables
solver.Clear()
fitness = solver.NumVar(0, solver.infinity(), 'fitness')
sol = [[]] * len(df)
objective_num = solver.Objective()
for i in range(0, len(df)):
    sol[i] = solver.IntVar(0, solver.infinity(), df['name'][i])
    objective_num.SetCoefficient(sol[i], 1)
objective_num.SetMinimization()
# cost = solver.NumVar(0, constraints['max_cost'], 'cost')
# size = solver.NumVar(0, constraints['max_size'], 'size')
# num = solver.IntVar(0, constraints['max_num'], 'num')

print('Number of variables:', solver.NumVariables())

Number of variables: 51


### create constraints
necessary:

cost $c$, size $s$ and num $n$


\begin{equation}
0 \leq c \leq c_{max} \qquad \in [0..20000] \\
0 \leq s \leq s_{max} \qquad \in [0..10] \\
0 \leq n \leq n_{max} \qquad \in \{ 0..5 \}
\end{equation}

In [108]:
# create constraints
# cost_c = solver.Constraint(0, constraints['max_cost'], 'cost')
# size_c = solver.Constraint(0, constraints['max_size'], 'size')
# num_c = solver.Constraint(0, constraints['max_num'], 'num')

# cost_c.SetCoefficient(cost, 1)
# size_c.SetCoefficient(size, 1)
# num_c.SetCoefficient(num, 1)
names_min = ['handling', 'positioning', 'joining']
names_max = ['cost', 'size']

const = [0] * (len(names_max) + len(names_min))
for i in range(len(names_max)):
    const[i] = solver.Constraint(0, constraints[names_max[i]], names_max[i])
    for j in range(len(df)):
        const[i].SetCoefficient(sol[j], float(df[names_max[i]][j]))
        

for i in range(len(names_min)):
    const[i] = solver.Constraint(constraints[names_min[i]], solver.infinity(), names_min[i])
    for j in range(len(df)):
        const[i].SetCoefficient(sol[j], float(df[names_min[i]][j]))

print('Number of constraints:', solver.NumConstraints())

Number of constraints: 5


## fitness function
\begin{equation}
f = \sum_{r \in \mathcal{R}} \frac{c_r}{C_m} + \frac{s_r}{S_m} \\
\text{with:} \\
\text{f = fitness} \\
\text{c = cost} \\
\text{s = size}
\end{equation}

In [36]:
def fitness(sol):
    f = sum(sol['cost'] / constraints['max_cost'] + sol['size'] / constraints['max_size'])
    return f

In [40]:
# create objective function, minimize fitness
# objective_fitness = solver.Objective()
# objective_fitness.SetCoefficient(const[0], 
# objective_fitness.SetMinimization()

In [173]:
solver.Solve()
print('Solution:')
print('Number of robots:', objective_num.Value())

cost = 0
size = 0

for i in range(len(sol)):
    if sol[i].solution_value() > 0: 
        c = df.loc[df.name==str(sol[i]), 'cost'].item()
        s = df.loc[df.name==str(sol[i]), 'size'].item()
        print(sol[i], '\tnum:', sol[i].solution_value(), '\tcost:', c, '€', '\tsize:', s, 'm2' )
        cost += c
        size += s

print('Cost =', cost, '€')
print('Size =', round(size, 3), 'm2')

Solution:
Number of robots: 9.257155061939345
robot6 	num: 4.194646162608577 	cost: 1466 € 	size: 0.84 m2
robot11 	num: 1.2571550619393437 	cost: 1769 € 	size: 1.01 m2
robot41 	num: 1.7428449380606559 	cost: 3612 € 	size: 2.81 m2
robot49 	num: 2.0625088993307688 	cost: 2585 € 	size: 0.15 m2
Cost = 9432 €
Size = 4.81 m2
